<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/OpenAI_Function_Calling_Tutorial_Assistents_API_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai -q

In [2]:
from openai import OpenAI
import json
import time
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="key",
)

Function to get order details

In [4]:
import requests
import json

def get_order_details(order_id):
    # URL of the API endpoint
    url = "http://44.203.99.19:8005/order_info_without_auth"

    # Parameters to be sent in the query string
    params = {'order_id': order_id}

    # Headers for the request
    headers = {'accept': 'application/json'}

    # Making the POST request
    response = requests.post(url, headers=headers, params=params)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parsing the JSON response
        result = response.json()
        return result['Result'][0]
    else:
        return f"Error: Unable to fetch order details. Status code: {response.status_code}"

order_details = get_order_details(order_id=4)
print(order_details)


available_functions = {
    "get_order_details": get_order_details,
}

{'order_id': 4, 'total_amount': 299.99, 'delivery_status': 'Shipped', 'current_location': 'Warehouse B', 'expected_delivery_date': '2023-01-18', 'customer_name': 'Alice Brown', 'product_id': 4, 'product_name': 'Tablet'}


In [8]:
functions = [
    {
        "name": "get_order_details",
        "description": "Retrieves the details of an order given its order ID.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "integer",
                    "description": "The unique identifier of the order.",
                }
            },
            "required": ["order_id"],
        },
    }
]

In [5]:
def get_gpt_response(messages):

  chat_completion = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      messages=messages,
      functions = functions,
      function_call="auto",
  )
  return chat_completion

Execute Function

In [6]:
def execute_function_call(function_name,arguments):
    function = available_functions.get(function_name,None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results

In [11]:
# query = "Tell me joke"
# query = "I want to know my order status"
query = "what is my order status of order id 4"
messages = [
    {
        "role": "user",
        "content": query
    }
]
response = get_gpt_response(messages)
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"order_id":4}', name='get_order_details'), tool_calls=None)


In [12]:
function_name = response.choices[0].message.function_call.name
arguments = response.choices[0].message.function_call.arguments
print(function_name,arguments)

get_order_details {"order_id":4}


In [13]:
function_response = execute_function_call(function_name,arguments)
function_response

{'order_id': 4,
 'total_amount': 299.99,
 'delivery_status': 'Shipped',
 'current_location': 'Warehouse B',
 'expected_delivery_date': '2023-01-18',
 'customer_name': 'Alice Brown',
 'product_id': 4,
 'product_name': 'Tablet'}

In [14]:
messages

[{'role': 'user', 'content': 'what is my order status of order id 4'}]

In [15]:
messages.append(response.choices[0].message)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": str(function_response),
    }
)
messages

[{'role': 'user', 'content': 'what is my order status of order id 4'},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"order_id":4}', name='get_order_details'), tool_calls=None),
 {'role': 'function',
  'name': 'get_order_details',
  'content': "{'order_id': 4, 'total_amount': 299.99, 'delivery_status': 'Shipped', 'current_location': 'Warehouse B', 'expected_delivery_date': '2023-01-18', 'customer_name': 'Alice Brown', 'product_id': 4, 'product_name': 'Tablet'}"}]

In [16]:
resposne = get_gpt_response(messages)
print(resposne.choices[0].message.content)

The order with ID 4 is for a Tablet with a total amount of $299.99. The order has been shipped and is currently at Warehouse B. The expected delivery date is 2023-01-18. The customer's name is Alice Brown.


Tools for assistent

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_order_details",
            "description": "Retrieves the details of an order given its order ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "integer",
                        "description": "The unique identifier of the order."
                    }
                },
                "required": ["order_id"]
            }
        }
    }
]

Create Assistent

In [18]:
assistant = client.beta.assistants.create(
  name="Ecommerce bot",
  instructions="You are an ecommerce bot. Use the provided functions to answer questions. Synthesise answer based on provided function output and be consise",
  model="gpt-4-1106-preview",
  tools = tools
)

Create Message inside Thread and Run

In [19]:
def create_message_and_run(assistant,query,thread=None):
  if not thread:
    thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=query
  )
  run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
  )
  return run,thread

Get Deatils of functions to be called

In [20]:
def get_function_details(run):

  print("\nrun.required_action\n",run.required_action)

  function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
  arguments = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
  function_id = run.required_action.submit_tool_outputs.tool_calls[0].id

  print(f"function_name: {function_name} and arguments: {arguments}")

  return function_name, arguments, function_id

Sumit Function Response

In [21]:
def submit_tool_outputs(run,thread,function_id,function_response):
    run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
      {
        "tool_call_id": function_id,
        "output": str(function_response),
      }
    ]
    )
    return run

In [22]:
query = "I want to know my order status"
run,thread = create_message_and_run(assistant=assistant,query=query)

In [23]:
run

Run(id='run_HVzBWoqZJmLb4XDgbT1klfSh', assistant_id='asst_kSjlvDlOuVAP1qRCQ8HrEfmW', cancelled_at=None, completed_at=None, created_at=1700341647, expires_at=1700342247, failed_at=None, file_ids=[], instructions='You are an ecommerce bot. Use the provided functions to answer questions. Synthesise answer based on provided function output and be consise', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_a28MVsGD8GZUhhUCVCPO8uHt', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_order_details', parameters={'type': 'object', 'properties': {'order_id': {'type': 'integer', 'description': 'The unique identifier of the order.'}}, 'required': ['order_id']}, description='Retrieves the details of an order given its order ID.'), type='function')])

In [24]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print("run status", run.status)

    if run.status=="requires_action":

        function_name, arguments, function_id  = get_function_details(run)

        function_response = execute_function_call(function_name,arguments)

        run = submit_tool_outputs(run,thread,function_id,function_response)

        continue
    if run.status=="completed":

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)

        user_input = input()
        if user_input == "STOP":
          break

        run,thread = create_message_and_run(assistant=assistant,query=user_input,thread=thread)

        continue;
    time.sleep(1)

run status completed
To help you with your order status, I will need the order ID. Please provide the order ID associated with the purchase you are inquiring about.
order id is 1
run status queued
run status requires_action

run.required_action
 RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_SOy0g26dzHmNG6HS4qobEUvk', function=Function(arguments='{"order_id":1}', name='get_order_details'), type='function')]), type='submit_tool_outputs')
function_name: get_order_details and arguments: {"order_id":1}
run status queued
run status in_progress
run status in_progress
run status completed
The order with ID 1 has the following status:

- Delivery Status: Shipped
- Current Location: Warehouse A
- Expected Delivery Date: 2023-01-10

The order consists of a laptop, and the total amount for this order is $999.99. If you have any more questions or need further assistance, feel free to ask!
when can i expect delivery
run status 